In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
! pip install autopep8

## Install FastAI V2

In [ ]:
! pip install -U fastai2

## Import fastAI v2 vision modules

In [ ]:
from fastai2.vision.all import *

## Semantic segmentation with UNet model

Recall the UNet model:
<img src="images/od/unet.png" height="800" width="800">

Download the dataset.
#### Note Here I'll download the "tiny" version, but you should switch it with 
```python
untar_data(URLs.CAMVID)
```

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

Let's look into the data

In [ ]:
path.ls()

In [ ]:
(path / 'images').ls()

In [ ]:
(path / 'labels').ls()

Load the class codes:

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str)
codes

In [ ]:
fnames = get_image_files(path/"images")
fnames[0]

In [ ]:
?? get_image_files

Initialize labels

In [ ]:
def label_func(fn): 
    return path/"labels"/f"{fn.stem}_P{fn.suffix}"

Initialize the data loader

In [ ]:
dls = SegmentationDataLoaders.from_label_func(path, bs=8, fnames=fnames, label_func=label_func, codes=codes, 
                                              seed=2020)

Look into the actual images

In [ ]:
dls.show_batch(max_n=8)

## Initialize the model

In [ ]:
learn = unet_learner(dls, resnet34)

Look in the model

In [ ]:
learn.model

Recall pixel-suffle up-sumpling layer:
<img src="images/od/pixel_shuffle_1.jpg" height="1000" width="1000">

Where are the concatenations?
<br/>
For this FastAi library uses so called runtime hooks

In [ ]:
print(learn.__dict__)

Check the optimizer

In [ ]:
learn.opt_func

Check the loss function:

In [ ]:
learn.loss_func

In [ ]:
?? FlattenedLoss

Check the type

In [ ]:
type(learn.loss_func)

In [ ]:
?? CrossEntropyLossFlat

In [ ]:
?? BaseLoss

In [ ]:
learn.fine_tune(8)

Check the results

In [ ]:
learn.show_results(max_n=6, figsize=(7, 8))

#### Use data block API for image loading

In [ ]:
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items = get_image_files,
                   get_y = label_func,
                   splitter=RandomSplitter(),
                   batch_tfms=aug_transforms(size=(120,160)))

In [ ]:
dls = camvid.dataloaders(path/"images", path=path, bs=8)

In [ ]:
dls.show_batch(max_n=8)

In [ ]:
?? get_image_files

In [ ]:
?? get_files

## Find learning rate

In [ ]:
learn.lr_find()

fine-tuning means it will update all layers weights and biases with discriminative learning rate

In [ ]:
?? learn.fine_tune

#### Task: Try train with different size (progressive growing) and different augmentations